# Как научить компьютер читать?

В этой тетрадке мы обучим свой w2v на википедии, а ещё возьмём чужой. Будем сравнивать эти две модели между собой.

word2vec был разработан группой исследователей Google в 2013 году, руководителем проекта был Томаш Миколов (сейчас работает в Facebook). Вот две самые главные статьи:

* [Efficient Estimation of Word Representations in Vector Space](https://arxiv.org/pdf/1301.3781.pdf)
* [Distributed Representations of Words and Phrases and their Compositionality](https://arxiv.org/abs/1310.4546)


## 1. Подготовка и обучение

Будем обучать w2v модель на википедии. К счастью, в её случае для всех языков предусмотрена система дампов. С [удобной странички](https://dumps.wikimedia.org) можно скачать текущую полную версию википедийного текста на любом языке. Например, [на русском.](https://dumps.wikimedia.org/ruwiki/).

Для обучения модели будем использовать библиотеку `gensim`. В ней уже есть удобный модуль доя работы с википедийными дампами, а также готовая хорошая реализация w2v-сетки. 

Код, скачивающий dump Википедии, был заимствован с [GitHub](https://github.com/lintseju/word_embedding) и затем упрощен 

In [1]:
!pip install gensim==4.1.2

In [2]:
DOWNLOAD = False
TRAIN_BI_TRI_GRAM = False

In [ ]:
from tqdm.auto import tqdm
import os
import requests
import math


def download_file(url, file):
    resp = requests.get(url, stream=True)

    total_size = int(resp.headers.get('content-length', 0))
    block_size = 1024
    wrote = 0
    with open(file, 'wb') as f:
        for data in tqdm(resp.iter_content(block_size), 
                         total=math.ceil(total_size / block_size), 
                         unit='KB',
                         unit_scale=True):
            wrote = wrote + len(data)
            f.write(data)


def download_wiki_dump(lang, path):
    url = f'https://dumps.wikimedia.org/{lang}wiki/latest/{lang}wiki-latest-pages-articles-multistream.xml.bz2'
    if not os.path.exists(path):
        download_file(url.format(lang=lang), path)
    else:
        print(f'{path} exists, skip download')
        raise SystemExit

if DOWNLOAD:
    os.makedirs('data', exist_ok=True)
    lang = 'ru'
    download_wiki_dump(lang, f'data/{lang}wiki.xml.bz2')

  0%|          | 0.00/4.51M [00:00<?, ?KB/s]

In [4]:
# у меня дамп википедии лежал вот тут:

lang = 'ru'
path = f'data/{lang}wiki.xml.bz2'

Для работы с текстами мы будем пользоваться библиотекой `gensim`. Она настолько хороша, что в ней есть даже специальные функции по работе с дампами с Википедии. Например, мы будем пользоваться для оценки модели специальным генератором, который будет считывать тексты с жёсткого диска по мере необходимости и не будет захламлять нам память. 

In [5]:
from gensim.corpora.wikicorpus import WikiCorpus

# для доступа к текстам мы будем пользоваться генератором wiki.get_texts()
wiki = WikiCorpus(path, dictionary=False)

Корпус википедии, оказавшийся в наших руках уже прошёл очистку от мусора и был токенезирован. Про то, как обычно тексты чистят и предобрабатывают можно почитать [вот тут.](https://github.com/DmitrySerg/OpenData/blob/master/RussianElections2018/Part_2_data_preparation.ipynb) Посмотрим на первые $40$ слов самой первой её статьи. 

In [ ]:
i = 0
for text in wiki.get_texts( ):
    i+=1
    if i == 2:
        break
    else:
        print(text[:40])

['литва', 'официальное', 'название', 'лито', 'вская', 'респу', 'блика', 'государство', 'расположенное', 'северной', 'части', 'европы', 'площадь', 'км²', 'протяжённость', 'севера', 'на', 'юг', 'км', 'запада', 'на', 'восток', 'км', 'население', 'составляет', 'человек', 'январь', 'занимает', 'место', 'мире', 'по', 'численности', 'населения', 'по', 'территории', 'имеет', 'выход', 'балтийскому', 'морю', 'расположена']


Видно, что многие слова оказались битыми. Например, слово "литовская" развалилось на "лито" и "вская". Предобработка была сделана не очень аккуратно. Тем не менее, у нас очень большой корпус документов. Закон больших чисел разрешает проигнорировать такие косяки. 

Попробуем выделить в тексте основные биграммы. Будем рассматривать их в дальнейшем как цельные токены. Существует целый ряд алгоритмов, занимающихся этим. Обычно все они сводятся к поиску вероятностей совместного появления двух слов в тексте. 

Код ниже, для большого корпуса текстов, будет работать довольно долго, но зато на выходе мы получим словарь биграмм, который мы сможем впоследствии применять к потоку текстов командой `bigram_transformer`. 

In [11]:
from gensim.models.phrases import Phrases, Phraser

if not TRAIN_BI_TRI_GRAM:
    bigram_transformer = Phraser.load('wiki_bigramm')
    trigram_transformer = Phraser.load('wiki_trigramm')

    # генератор текстов с биграммами
    def text_generator_bigram():
        for text in wiki.get_texts():
            yield bigram_transformer[[word for word in text]]

    def text_generator_trigram():
        for text in wiki.get_texts():
            yield trigram_transformer[bigram_transformer[[word for word in text]]]

**Parameters for `Phrases`:**


* `sentences` (iterable of list of str, optional) – The sentences iterable can be simply a list, but for larger corpora, consider a generator that streams the sentences directly from disk/network, See BrownCorpus, Text8Corpus or LineSentence for such examples.

* `min_count` (float, optional) – Ignore all words and bigrams with total collected count lower than this value.

* `threshold` (float, optional) – Represent a score threshold for forming the phrases (higher means fewer phrases). A phrase of words a followed by b is accepted if the score of the phrase is greater than threshold. Heavily depends on concrete scoring-function, see the scoring parameter.

* `delimiter` (str, optional) – Glue character used to join collocation tokens.

* `scoring` ({'default', 'npmi', function}, optional) –

    Specify how potential phrases are scored. scoring can be set with either a string that refers to a built-in scoring function, or with a function with the expected parameter names. Two built-in scoring functions are available by setting scoring to a string:

    - ”default” - original_scorer().\
$score(i, j) = \dfrac{n_{ij} - δ}{n_i\cdot n_j}$,\
$n_{ij}$ - число вхождений биграммы\
$n_i$ и $n_j$ - число вхлждений слов по $i$ и $j$ по отдельности\
$δ$ - небольшая константа
    - ”npmi” - npmi_scorer().

* `connector_words` (set of str, optional) –

    Set of words that may be included within a phrase, without affecting its scoring. No phrase can start nor end with a connector word; a phrase may contain any number of connector words in the middle.

    If your texts are in English, set connector_words=phrases.ENGLISH_CONNECTOR_WORDS.

    This will cause phrases to include common English articles, prepositions and conjuctions, such as bank_of_america or eye_of_the_beholder.

    For other languages or specific applications domains, use custom connector_words that make sense there: connector_words=frozenset("der die das".split()) etc.

In [ ]:
%%time
from gensim.models.phrases import Phrases, Phraser

# хочется посмотреть на самые частые биграммы и использовать их при обучении как токены
bigram = Phrases(wiki.get_texts())
bigram_transformer = Phraser(bigram)

bigram_transformer.save('wiki_bigramm')

# генератор текстов с биграммами
def text_generator_bigram():
    for text in wiki.get_texts():
        yield bigram_transformer[[word for word in text]]

CPU times: user 1h 11min 58s, sys: 3min 7s, total: 1h 15min 5s
Wall time: 3h 27min


Посмотрим что нам будет выдавать такой генератор на примере первой статьи с википедии.

In [ ]:
i = 0
for item in text_generator_bigram( ):
    i +=1 
    if i == 2:
        break
    else:
        print(item[:20])

['литва', 'официальное_название', 'лито_вская', 'респу_блика', 'государство', 'расположенное', 'северной', 'части', 'европы', 'площадь_км²', 'протяжённость_севера', 'на', 'юг', 'км', 'запада', 'на', 'восток', 'км', 'население', 'составляет']


Хорошая новость: мы поправили некоторые косяки предобработки и скрепили слово "литовская" в единое целое. Другой вопрос в том, что у нас в корпусе теперь есть слово "лито_вская" и "литовская". Алгоритм будет думать, что это разные слова. Интересно будет в конце посмотреть насколько близки будут их вектора. 

Кроме всего прочьего, у нас в выборке появились и настояшие биграммы. Например, "площадь_км²". По аналогии можно соорудить код для поиска самых частых триграмм. Например, триграммой будет словосочетание "по моему мнению" или "вторая мировая война".

In [ ]:
%%time
trigram = Phrases(text_generator_bigram())
trigram_transformer = Phraser(trigram)

trigram_transformer.save('wiki_trigramm')

def text_generator_trigram():
    for text in wiki.get_texts():
        yield trigram_transformer[bigram_transformer[[word for word in text]]]

CPU times: user 1h 57min 8s, sys: 3min 22s, total: 2h 31s
Wall time: 4h 2min 41s


In [ ]:
i = 0
for item in text_generator_trigram( ):
    i +=1 
    if i == 2:
        break
    else:
        print(item[:20])

['литва', 'официальное_название', 'лито_вская', 'респу_блика_государство', 'расположенное', 'северной_части', 'европы', 'площадь_км²', 'протяжённость_севера', 'на', 'юг', 'км', 'запада', 'на', 'восток', 'км', 'население', 'составляет', 'человек', 'январь']


На этом давайте закончим нашу подготовку и попробуем собрать и обучить модель. Конечно же учиться она будет довольно долго. Не факт, что на слабом компьютере она вообще выучится. 

In [ ]:
%%time 
from gensim.models.word2vec import Word2Vec

# теперь сама модель
# size - размерность векторов, которые мы хотим обучить
# window - ширина окна контекста
# min_count - если слово встречается реже, для него не учим модель
model = Word2Vec(vector_size=300, window=7, min_count=10, workers=-1)

# строительство словаря, чтобы обучение шло быстрее
model.build_vocab(text_generator_trigram())

CPU times: user 1h 54s, sys: 2min 41s, total: 1h 3min 35s
Wall time: 3h 13min 32s


In [ ]:
model.save('wiki_model_vocab')

In [13]:
class SentencesIterator():
    """
    Source: https://jacopofarina.eu/posts/gensim-generator-is-not-iterator/
    """
    def __init__(self, generator_function):
        self.generator_function = generator_function
        self.generator = self.generator_function()

    def __iter__(self):
        # reset the generator
        self.generator = self.generator_function()
        return self

    def __next__(self):
        result = next(self.generator)
        if result is None:
            raise StopIteration
        else:
            return result

sentIter = SentencesIterator(text_generator_trigram)

In [14]:
%%time

from gensim.models.word2vec import Word2Vec

# обучение модели 
# первый аргумент - наша выборка, генератор будет вкидывать в модель наши тексты, пока они не кончатся
# второй аргумент - число примеров в выборке 
# третий аргумент - количество эпох обучения: сколько раз модель пройдётся по всему корпусу текстов

model = Word2Vec(vector_size=300, window=7, min_count=10, workers=-1)
model = Word2Vec.load('wiki_model_vocab')

model.train(sentIter, total_examples=model.corpus_count, epochs=10)

# # сохраним обученную модель
# model.save('wiki_model')

CPU times: user 41.3 s, sys: 9.14 s, total: 50.5 s
Wall time: 3min 7s


Обученную модель можно сохранить. Процесс её обучения довольно трудоёмок, не очень хочется его повторять по несколько раз.

In [ ]:
# сохраним обученную модель
model.save('wiki_model')

По аналогии можно сохранить создатель триграмм. Он тоже учился довольно долго, а нам хотелось бы в дальнейшем его переиспользовать.

## 2. Изучаем свойства модели

Итак, самая сложная часть оказалась позади. Модель обучено. Теперь пришло время немного поисследовать свойства, которыми модель обладает. Попробуем посмотреть на линейные соотношения между векторами, соотвествующими тем или иным слова в получившемся семантическом пространстве. 

Скачаем одну из моделей с проекта [rusvectores](https://rusvectores.org/ru/models/) и сравним с ней свойства нашей модели. Возьмём модель с гордым именем `ruwikiruscorpora_upos_skipgram_300_2_2018`, обученую на корпусе Википедии и НКРЯ (национальный корпус русского языка) в декабре 2017 года. 

W2V модели учат для самых различных целей. Какие-то из корпусов лемматизируют, какие нет. Мы, обучая модель на википедии, не делали лемматизацию. В случае rusvec-модели лемматизация была сделана. Более того, их модель училась на больших объёмах данных, чем наша. Также в ней выделены различные части речи, присущие словам. 

Модели word2vec бывают разных форматов:

* `.vec.gz` — обычный файл
* `.bin.gz` — бинарник

Загружаются они с помощью одного и того же класса `KeyedVectors`, меняется только параметр `binary` у функции `load_word2vec_format`. 

Если же эмбеддинги обучены **не** с помощью word2vec, то для загрузки нужно использовать функцию `load`. Т.е. для загрузки предобученных эмбеддингов *glove, fasttext, bpe* и любых других нужна именно она.

In [ ]:
import gensim

# подгрузим обученную модель, если вдруг мы сбросили скрипт
our_model = gensim.models.Word2Vec.load('wiki_model')

In [ ]:
import gensim

# подгрузим модель, обученную ребятами из rusvectores 
rv_name = 'models/ruwikiruscorpora_upos_skipgram_300_2_2018.vec.gz'
rusvec_model = gensim.models.KeyedVectors.load_word2vec_format(rv_name, binary=False)

Для начала посмотрим как выглядит слово в получившемся пространстве.

In [ ]:
# вектор слова
our_model.wv['король'][:10]

array([ 0.4187494 ,  1.3197291 ,  0.5561069 , -2.674209  , -0.39070314,
       -1.7381759 ,  2.4423337 ,  1.1509248 ,  0.49805412,  1.0481958 ],
      dtype=float32)

Помотрим на размерность вектора в рамках нашей модели.

In [ ]:
our_model.wv['король'].shape

(300,)

Посмотрим на размерность вектора в рамках rusvec модели. При оценивании своих моделей ребята пытаются уточнять в разметке части речи для всех слов с помощью своих разметок и специальных алгоритмов. Если вам вдруг захотелось тоже протэгировать частьми речи слова, `pymorty2` и `pymystem` умеют это делать. 

In [ ]:
rusvec_model.wv['король_NOUN'].shape

(300,)

### 2.1 Похожесть слов

Можно посмотреть насколько различные слова похожи между собой в семантическом плане. Похожесть между словами считается с помощью косинусной метрики. 

In [ ]:
# наша модель
print('грязный и вонючий:', our_model.wv.similarity('грязный', 'вонючий'))
print('грязный и чистый:', our_model.wv.similarity('грязный', 'чистый'))
print('грязный и грязный:', our_model.wv.similarity('грязный', 'грязный'))

грязный и вонючий: 0.7032279588356054
грязный и убранный: 0.6671076378477605
грязный и грязный: 1.0000000000000004


In [ ]:
# rusvec модель
print('грязный и вонючий:', rusvec_model.wv.similarity('грязный_ADJ', 'вонючий_ADJ'))
print('грязный и чистый:', rusvec_model.wv.similarity('грязный_ADJ', 'чистый_ADV'))
print('грязный и грязный:', rusvec_model.wv.similarity('грязный_ADJ', 'грязный_ADJ'))

грязный и вонючий: 0.6302576004741985
грязный и убранный: 0.3905526637350729
грязный и грязный: 1.0


### 2.2 Самые близкие слова

Можно посмотреть на самые близкие слова к какому-то конкретному слову. Посмотрим на самые близкие слова к слову грязный.

In [ ]:
our_model.most_similar('грязный')

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('шумный', 0.824225902557373),
 ('грустный', 0.8109831213951111),
 ('жуткий', 0.8098311424255371),
 ('ужасный', 0.8051483631134033),
 ('тёмный', 0.7896004915237427),
 ('темный', 0.7869464755058289),
 ('нежный', 0.7816108465194702),
 ('жирный', 0.7760384678840637),
 ('весёлый', 0.774817705154419),
 ('громкий', 0.7737864255905151)]

In [ ]:
rusvec_model.most_similar('грязный_ADJ')

[('грязноватый_ADJ', 0.660734236240387),
 ('грязнейший_ADJ', 0.6599724292755127),
 ('неопрятный_ADJ', 0.634252667427063),
 ('вонючий_ADJ', 0.6302576065063477),
 ('запачкать_VERB', 0.61855149269104),
 ('заплеванный_VERB', 0.6133441925048828),
 ('запачканный_VERB', 0.6034902334213257),
 ('испачкать_ADJ', 0.5984973907470703),
 ('загаженный_VERB', 0.5979785919189453),
 ('мерзкий_ADJ', 0.5978673696517944)]

Видно, что чужая модель выгодно отличается от нашей. Посмотрим ещё на пару примеров. Нас интересует дружелюбие.

In [ ]:
our_model.most_similar('дружелюбный')

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('неуклюжий', 0.8484416007995605),
 ('добродушный', 0.846727728843689),
 ('эгоистичный', 0.8443366289138794),
 ('обаятельный', 0.8425811529159546),
 ('вспыльчивый', 0.8413093686103821),
 ('самоуверенный', 0.8410248756408691),
 ('жизнерадостный', 0.8406734466552734),
 ('застенчивый', 0.8355358839035034),
 ('вежливый', 0.8343687653541565),
 ('наивный', 0.8318225145339966)]

In [ ]:
rusvec_model.most_similar('дружелюбный_ADJ')

[('доброжелательный_ADJ', 0.7317343354225159),
 ('добродушный_ADJ', 0.6525911092758179),
 ('приветливый_ADJ', 0.6513131856918335),
 ('уживчивый_ADJ', 0.6482845544815063),
 ('коммуникабельный_ADJ', 0.6472278833389282),
 ('общительный_ADJ', 0.6395638585090637),
 ('неконфликтный_ADJ', 0.6390734910964966),
 ('доброжелательной_ADJ', 0.632719874382019),
 ('приветливый_NOUN', 0.6252726316452026),
 ('миролюбивый_ADJ', 0.6205745935440063)]

Нас интересует Шок.

In [ ]:
our_model.most_similar('шок')

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('бред', 0.7624124884605408),
 ('психоз', 0.7540462017059326),
 ('стресс', 0.7503001689910889),
 ('страх', 0.7487086057662964),
 ('головокружение', 0.7479767799377441),
 ('обморок', 0.7372667193412781),
 ('ступор', 0.7199352979660034),
 ('боли', 0.716423511505127),
 ('раздражение', 0.7102159857749939),
 ('оргазм', 0.7092399001121521)]

In [ ]:
rusvec_model.most_similar('шок_NOUN')

[('шоко_NOUN', 0.6282049417495728),
 ('гиповолемический_ADJ', 0.6145074963569641),
 ('шоковой_ADJ', 0.6083506345748901),
 ('кардиогенный_ADJ', 0.5890294313430786),
 ('обморок_NOUN', 0.5806757807731628),
 ('коматозный_ADJ', 0.5661958456039429),
 ('анафилактический_ADJ', 0.556747555732727),
 ('инсульта_NOUN', 0.5564901232719421),
 ('стресс_NOUN', 0.5562987923622131),
 ('анафилактоидный_ADJ', 0.5518753528594971)]

Во многих вещах наши модели согласны друг с другом. Движемся дальше! Настал черёд арифметики.

### 2.3 Арифметика

Попробуем провернуть первое уравнение. 

$$ Король + Женшина - Мужчина = \quad ???$$


In [ ]:
our_model.most_similar(positive=['женщина', 'король'], negative=['мужчина'])[:5]

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('королева', 0.6225535869598389),
 ('империя', 0.5604485869407654),
 ('принцесса', 0.5506317615509033),
 ('императрица', 0.5310197472572327),
 ('король_ок_ок', 0.5229284763336182)]

In [ ]:
rusvec_model.most_similar(positive=['женщина_NOUN', 'король_NOUN'], 
                             negative=['мужчина_NOUN'])[:5]

[('королева_NOUN', 0.7153134346008301),
 ('королева_ADV', 0.6489790678024292),
 ('король_PROPN', 0.5975136756896973),
 ('королева_ADJ', 0.5909769535064697),
 ('короля_NOUN', 0.5825802087783813)]

$$ Москва + Франция - Россия = \quad ???$$

In [ ]:
our_model.most_similar(positive=['москва', 'франция'], negative=['россия'])[:5]

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('париж', 0.5632048845291138),
 ('жан', 0.5306471586227417),
 ('жак', 0.5089118480682373),
 ('пьер', 0.5062865018844604),
 ('французский', 0.5051255822181702)]

In [ ]:
rusvec_model.most_similar(positive=['москва_NOUN', 'франция_NOUN'], negative=['россия_NOUN'])[:5]

[('париж_NOUN', 0.4464000165462494),
 ('италия_NOUN', 0.4293068051338196),
 ('брюссель_NOUN', 0.4278932809829712),
 ('швеция_NOUN', 0.4128666818141937),
 ('англия_NOUN', 0.40511107444763184)]

$$ Математик + Женшина - Мужчина = \quad ???$$

In [ ]:
our_model.most_similar(positive=['математик', 'женщина'], negative=['мужчина'])[:5]

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('филолог', 0.6791591048240662),
 ('получившая_степень_доктора', 0.665369987487793),
 ('лингвист', 0.6601260900497437),
 ('доктор_философии', 0.6564381122589111),
 ('доктор_филологических_наук', 0.6553531885147095)]

In [ ]:
rusvec_model.most_similar(positive=['математик_NOUN', 'женщина_NOUN'], negative=['мужчина_NOUN'])[:5]

[('физик_NOUN', 0.6517059803009033),
 ('физик-теоретик_NOUN', 0.6240127086639404),
 ('философ_NOUN', 0.6040728688240051),
 ('физико-химик_NOUN', 0.597895622253418),
 ('геометр_NOUN', 0.5913760662078857)]

Почему-то наша модель оказалась сексистом... Это артефакт выборки. Запомните про это, ниже мы обсудим артефакты подробнее. 

$$ Человек - Животное = \quad ???$$

In [ ]:
our_model.most_similar(positive=['человек'],negative=['животное'])[:5]

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


[('тысяч_человек', 0.4908541440963745),
 ('тыс_человек', 0.4347049593925476),
 ('райкомы', 0.38436031341552734),
 ('делегатов', 0.3816929757595062),
 ('штатных_сотрудников', 0.3813714385032654)]

In [ ]:
rusvec_model.most_similar(positive=['человек_NOUN'],negative=['животное_NOUN'])[:5]

[('чел[овек_NOUN', 0.3204188346862793),
 ('человек_PROPN', 0.299196720123291),
 ('человѣкъ_PROPN', 0.29532384872436523),
 ('человеколо_NOUN', 0.2849539816379547),
 ('человек_VERB', 0.28311628103256226)]

При обучении н английском корпусе слов, можно было бы уведить, что `Human - Animal = Ethics`.

In [ ]:
our_model.most_similar(positive=['президент'],negative=['мощь'])[:5]

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('вице_президент', 0.5562783479690552),
 ('дмитрий_медведев', 0.5472823977470398),
 ('спикер', 0.5424843430519104),
 ('михаил_маргелов', 0.5396710634231567),
 ('вице_премьер', 0.5389058589935303)]

In [ ]:
rusvec_model.most_similar(positive=['президент_NOUN'],negative=['мощь_NOUN'])[:5]

[('президент_PROPN', 0.5648089647293091),
 ('вице-президент_NOUN', 0.5592658519744873),
 ('экс-президент_NOUN', 0.44517824053764343),
 ('экс-президентый_NOUN', 0.4248402714729309),
 ('премьер-министр_NOUN', 0.41906532645225525)]

И снова для русской википедии мы наблюдаем забавный артефакт :) 

In [ ]:
our_model.most_similar(positive=['летучая_мышь','брюс_уэйн'])[:5]

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('пиноккио', 0.8352062702178955),
 ('женщина_кошка', 0.8283794522285461),
 ('капитан_крюк', 0.8133963346481323),
 ('джинн', 0.8126029968261719),
 ('дракула', 0.809827446937561)]

In [ ]:
rusvec_model.most_similar(positive=['мышь_NOUN','летучий_ADJ','брюс_NOUN'])[:5]

[('листоносый_ADJ', 0.6629314422607422),
 ('мышью_NOUN', 0.6600611209869385),
 ('крыса_NOUN', 0.6554200649261475),
 ('micromys_PROPN', 0.641371488571167),
 ('мыши_NOUN', 0.6350268721580505)]

К сожалению в выбранной мною для сравнения модели нет биграмм :( 

In [ ]:
our_model.most_similar(positive=['питер_паркер','паук'])[:10]

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('гоблин', 0.8566164970397949),
 ('стервятник', 0.8511500358581543),
 ('росомаха', 0.850637674331665),
 ('веном', 0.8489575982093811),
 ('мутант', 0.845893383026123),
 ('циклоп', 0.8411886692047119),
 ('киборг', 0.8410425186157227),
 ('инопланетянин', 0.8373299241065979),
 ('злодей', 0.8354233503341675),
 ('пришелец', 0.8316423296928406)]

In [ ]:
rusvec_model.most_similar(positive=['питер_NOUN','паук_NOUN'])[:10]

[('паук_PROPN', 0.6240770816802979),
 ('-паука_NOUN', 0.590177059173584),
 ('осьминог_PROPN', 0.5895593762397766),
 ('человек-паук_NOUN', 0.5872828960418701),
 ('гарри::озборн_PROPN', 0.584584653377533),
 ('питер_PROPN', 0.5779144167900085),
 ('-паук_NOUN', 0.5753026604652405),
 ('-паука_X', 0.5728250741958618),
 ('человек-паук_PROPN', 0.5703564882278442),
 ('хеллбой_NOUN', 0.5701111555099487)]

$$ Ваши \mbox{ } уравнения $$

In [ ]:
# Идеи: 
# 
# пицца - италия + сибирь
# шиншилла - мышь + собака .... 
#

### 2.4 Найди лишнее

Можно попросить модель найти лишнее слово в каком-нибудь векторе. 

In [ ]:
rusvec_model.doesnt_match('яблоко_NOUN груша_NOUN виноград_NOUN банан_NOUN лимон_NOUN картофель_NOUN'.split())

'картофель_NOUN'

In [ ]:
our_model.wv.doesnt_match('яблоко груша виноград банан лимон картофель'.split())

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.


'яблоко'

### 2.5 Взаимоотношения 

Если модель училась без лемматизации, можно попробовать увидеть не только то, что столицы взаимоотносятся с названиями стран одинаково, но и поймать более интересные эффекты.

![w2v](https://cdn-images-1.medium.com/max/2600/1*sXNXYfAqfLUeiDXPCo130w.png)

In [ ]:
our_model.most_similar(positive=['лондон', 'испания'], negative=['англия'])[0][0]

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


'мадрид'

Например, увидеть взаимотношения между единственным числои и множественным.

In [ ]:
our_model.most_similar(positive=['яблоки', 'грузовик'], 
                       negative=['яблоко'])[0][0]

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


'грузовики'

А также между разными степенями прилагательных.

In [ ]:
our_model.most_similar(positive=['красивый', 'страшная'], 
                       negative=['красивая'])[0][0]

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


'страшный'

Можно найти и многие другие интересные семантические свойства.

## 3. Артефакты

Нужно понимать, что в выборке, на основе которой вы обучали модель могут быть различные "артефакты". Например, если мы обучались на корпусе новостей, мы можем неожиданно обнаружить, что к Индонезии очень близко землятрясение. Почему? Да просто потому что в корпусе все статьи, связанные с Индонезией упоминали недавнее землятрясение. С такми артефактами приходится бороться и переодически модель приходится переобучать. 

Одним из забавных артефактов, найденных в рамках нашей модели было то, что если отобрать у президнета мощь, получится Дмитрий Медведев. Вот другой пример подобного артефакта. Машина времени в контексте нашей модели это рок-группа. Никаких вещей, связанных с научной фантастикой не нарисовалось.

In [ ]:
model.most_similar('машина_времени')

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('король_шут', 0.8701559901237488),
 ('агата_кристи', 0.8659921288490295),
 ('сплин', 0.8545758724212646),
 ('весёлые_ребята', 0.8407142162322998),
 ('чайф', 0.8382944464683533),
 ('ногу_свело', 0.8371785879135132),
 ('ночные_снайперы', 0.8325160145759583),
 ('наив', 0.8310174942016602),
 ('валерий_леонтьев', 0.8306418657302856),
 ('андрей_макаревич', 0.8283897638320923)]

# Ссылки  да почиташки

* [Про w2v и русский сексизм](https://nikolenko.livejournal.com/267442.html)
* [Предобученная w2v для английского языка](https://code.google.com/archive/p/word2vec)
* [Неплохая заметка со ссылками на 5 базовых работ по w2v](https://blog.acolyer.org/2016/04/21/the-amazing-power-of-word-vectors/)
* [Rusvec-модели](https://rusvectores.org/ru/models/) и подробное описание проекта.
* [Предобработчик текстов](https://github.com/akutuzov/webvectors/blob/master/preprocessing/rusvectores_tutorial.ipynb) для rusvec моделей. 
* [Статья про w2v для Хабра и преходов по ссылкам + обучение моделей на них](https://nbviewer.jupyter.org/github/Yorko/mlcourse_open/blob/master/jupyter_russian/tutorials/word2vec_demonzheg.ipynb)